In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from model.data import get_patient_records, load_pic

ModuleNotFoundError: No module named 'model.data'

In [ ]:
ignore_variables = {
    'PATIENTS': ['ROW_ID', 'EXPIRE_FLAG', 'DOD', 'SUBJECT_ID'],
    'ADMISSIONS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID'],
    'ICUSTAYS': ['ROW_ID', 'HADM_ID', 'ICUSTAY_ID', 'SUBJECT_ID'],
    'SURGERY_INFO': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'VISIT_ID', 'OPER_ID'],
    'DIAGNOSES_ICD': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID'],
    'SURGERY_VITAL_SIGNS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'VISIT_ID', 'OPER_ID', 'ITEM_NO'],
    'EMR_SYMPTOMS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'EMR_ID', 'SYMPTOM_NAME_CN'],
    'LABEVENTS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'VALUEUOM', 'CHARTTIME'],
    'MICROBIOLOGYEVENTS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'AB_ITEMID', 'ORG_ITEMID', 'SPEC_ITEMID'],
    'OR_EXAM_REPORTS': ['ROW_ID', 'HADM_ID', 'SUBJECT_ID', 'REPORTTIME'],
    'CHARTEVENTS': ['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'VALUEUOM', 'STORETIME'],
    'INPUTEVENTS': ['ROW_ID', 'HADM_ID', 'ICUSTAY_ID', 'SUBJECT_ID'],
    'OUTPUTEVENTS': ['ROW_ID', 'HADM_ID', 'ICUSTAY_ID', 'ITEMID', 'SUBJECT_ID'],
    'PRESCRIPTIONS': ['ROW_ID', 'HADM_ID', 'ICUSTAY_ID', 'SUBJECT_ID', 'DOSE_UNIT_RX', 'PROD_STRENGTH', 
                      'DRUG_NAME_GENERIC', 'DOSE_UNIT_RX', 'DRUG_FORM', 'DRUG_NAME'],
    'D_ITEMS': ['ROW_ID', 'ITEMID', 'UNITNAME', 'LINKSTO', 'LABEL_CN', 'CATEGORY'],
    'D_LABITEMS': ['ROW_ID', 'ITEMID', 'LABEL_CN'],
    'D_ICD_DIAGNOSES': ['ROW_ID']
}

In [ ]:
es = load_pic()

In [ ]:
surgery_df = get_patient_records(es, "SURGERY_INFO", subject_id)
surgery_df

surgery_end = surgery_df["SURGERY_END_TIME"].iloc[0]
print(surgery_end)

In [ ]:
subject_id = 3779

prescription_df = get_patient_records(es, "PRESCRIPTIONS", subject_id)
prescription_df

In [ ]:
prescription_df = prescription_df[prescription_df["STARTDATE"] < surgery_end]
prescription_df

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

values, counts = np.unique(prescription_df["DRUG_NAME_EN"], return_counts=True)

x = np.arange(len(values))
y = counts

plt.bar(x, y)
plt.xticks(ticks=x, labels=values, rotation=45, ha='right')
plt.show()

In [ ]:
es["SURGERY_INFO"].df

In [ ]:
import sys

prescription_df = es["PRESCRIPTIONS"].df
values, counts = np.unique(prescription_df["DRUG_NAME_EN"], return_counts=True)

def exists(column, values=None):
    assert values is not None
    return [v in column.values for v in values]

target_entity = 'SURGERY_INFO'
label_times = es['SURGERY_INFO'].df[['UNI_OPER_ID', 'SURGERY_END_TIME']]
label_times.columns = ['instance_id', 'time']

Medications = ft.primitives.make_agg_primitive(function=exists,
                                 input_types=[ft.variable_types.Discrete],
                                 return_type=ft.variable_types.Boolean,
                                 number_output_features=len(values))
    
med_feature_matrix, med_features_list = ft.dfs(entityset=es,
                                       target_entity=target_entity,
                                       agg_primitives=[Medications(values=values)],
                                       trans_primitives=[],
                                       allowed_paths=[['SURGERY_INFO', 'ADMISSIONS'],  
                                                      ['SURGERY_INFO', 'ADMISSIONS', 'PRESCRIPTIONS']],
                                       ignore_variables=ignore_variables,
                                       cutoff_time=label_times,
                                       max_depth=2,
                                       verbose=True)

med_feature_matrix = med_feature_matrix.fillna(0)

In [ ]:
drug_feature_matrix = med_feature_matrix.copy()

for column in med_feature_matrix:
    if not "ADMISSIONS.EXISTS(PRESCRIPTIONS.DRUG_NAME_EN)" in column:
        drug_feature_matrix.drop(column, inplace=True, axis="columns")
        
drug_feature_matrix

In [ ]:
prescription_counts = drug_feature_matrix.sum(axis="rows")
prescription_counts